# 機械学習フロー
Kaggleの Home Credit Default Risk コンペティションを題材に、機械学習の実践的な流れを学びます。特に適切な 検証 を行い、高い 汎化性能 のあるモデルを完成させることを目指します。

## 【問題1】クロスバリデーション
事前学習期間では検証データをはじめに分割しておき、それに対して指標値を計算することで検証を行っていました。（ホールドアウト法）しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション（交差検証） を行います。分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割のためにscikit-learnにはKFoldクラスが用意されています。<br>
<br>
事前学習期間の課題で作成したベースラインモデルに対してKFoldクラスによるクロスバリデーションを行うコードを作成し実行してください。<br>
<br>
[sklearn.model_selection.KFold — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold)

In [1]:
import time

In [2]:
# メモリ使用量の節約関数
# ソースコードはkernelより取得
import numpy as np
import pandas as pd

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [51]:
#データの読み込み
print("train")
train_raw = import_data('/Users/tamiyagt/Documents/Machine Learning/02_Kaggle/home credit/application_train.csv')
print("test")
test_raw = import_data('/Users/tamiyagt/Documents/Machine Learning/02_Kaggle/home credit/application_test.csv')

np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 300)

train
Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 59.54 MB
Decreased by 79.2%
test
Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 9.40 MB
Decreased by 79.1%


>データの前処理として、前回の課題で行ったものを実装する。

In [52]:
# あとでデータ分割するために要素数を把握
ntrain = train_raw.shape[0]
ntest = test_raw.shape[0]

# 目的変数をyに格納
target = train_raw['TARGET']
# SK_ID_CURRをX_idに格納
sk_id = test_raw['SK_ID_CURR']

# trainとtestを結合
all_data = pd.concat([train_raw, test_raw]).reset_index(drop=True)
all_data.drop(['SK_ID_CURR', 'TARGET'], axis=1, inplace=True)

In [53]:
# 欠損率の確認
def get_miss(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)

    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    mis_val_table = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'}
    )

    # 表を欠損率順に並べ替え、欠損値０のサンプルは除外
    mis_val_table_sort = mis_val_table[
        mis_val_table.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    
    return mis_val_table_sort

miss_data = get_miss(all_data)

print("欠損値を含む列数：{}".format(miss_data.shape[0]))
print("-----------------")
print(miss_data)

欠損値を含む列数：67
-----------------
                              Missing Values  % of Total Values
COMMONAREA_MEDI                       248360               69.7
COMMONAREA_AVG                        248360               69.7
COMMONAREA_MODE                       248360               69.7
NONLIVINGAPARTMENTS_MEDI              246861               69.3
NONLIVINGAPARTMENTS_MODE              246861               69.3
NONLIVINGAPARTMENTS_AVG               246861               69.3
FONDKAPREMONT_MODE                    243092               68.2
LIVINGAPARTMENTS_MODE                 242979               68.2
LIVINGAPARTMENTS_MEDI                 242979               68.2
LIVINGAPARTMENTS_AVG                  242979               68.2
FLOORSMIN_MODE                        241108               67.7
FLOORSMIN_MEDI                        241108               67.7
FLOORSMIN_AVG                         241108               67.7
YEARS_BUILD_MODE                      236306               66.3
YEARS_BUIL

In [54]:
# 補完する変数を特定
features = ['EXT_SOURCE_1','EXT_SOURCE_2', 'EXT_SOURCE_3','NAME_TYPE_SUITE',
            'DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',
            'OBS_30_CNT_SOCIAL_CIRCLE','AMT_GOODS_PRICE','AMT_ANNUITY','CNT_FAM_MEMBERS',
            'DAYS_LAST_PHONE_CHANGE']

# カテゴリー変数を特定
categorical = all_data[features].select_dtypes(exclude='number')
# 数値変数を特定
numerical = all_data[features].select_dtypes(include='number')

# カテゴリー変数を最頻値で補完
for f in categorical:
    all_data[f].fillna(all_data[f].mode()[0], inplace=True)

# # 数値変数を中央値で補完
for f in numerical:
    all_data[f].fillna(all_data[f].median(), inplace=True)    

    
# 欠損値10%以上の列を削除
all_data_drop = all_data.dropna(axis=1, thresh=350000)

# データを確認
miss_data = get_miss(all_data_drop)

print("欠損値を含む列数：{}".format(miss_data.shape[0]))
print("-----------------")
print(miss_data)

欠損値を含む列数：0
-----------------
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [167]:
# LabelEncodingを行う

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in all_data_drop:
    if all_data_drop[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(all_data_drop[col].unique())) <= 2:
            # Fit the data
            le.fit(all_data_drop[col])
            # Transform both all_data
            all_data_drop[col] = le.transform(all_data_drop[col])
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('{} columns were label encoded.'.format(le_count))

# One-Hot Encodingを行う
# ソースコードはKaggleのkernelから取得

# one-hot encoding of categorical variables
all_data_lab = pd.get_dummies(all_data_drop)

print('Data shape: ', all_data_lab.shape)

0 columns were label encoded.
Data shape:  (356255, 160)


In [168]:
from sklearn.preprocessing import PolynomialFeatures

all_data_lab = all_data_lab.copy()

# DTIを追加
all_data_lab['DTI'] = all_data_lab['AMT_CREDIT']/all_data_lab['AMT_INCOME_TOTAL']
# DTI計算の元となった特徴量を削除
all_data_lab.drop(columns=['AMT_CREDIT', 'AMT_INCOME_TOTAL'], inplace=True)

# 多次元特徴量の生成
poly = PolynomialFeatures(include_bias=False)

all_data_poly = poly.fit_transform(
    all_data_lab.loc[:, ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DTI', 'EXT_SOURCE_2', 'EXT_SOURCE_3']])
feature_names = poly.get_feature_names(
    input_features = ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DTI', 'EXT_SOURCE_2', 'EXT_SOURCE_3'])

# # 多次元特徴量生成の元となった特徴量を削除（重複するため）
all_data_lab.drop(columns=['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DTI', 'EXT_SOURCE_2', 'EXT_SOURCE_3'], inplace=True)

# # 元のデータフレームと多次元特徴量を結合
all_data_lab = pd.concat((all_data_lab,
                             pd.DataFrame(data=all_data_poly, columns=feature_names)),
                             axis=1)

# trainとtestに分割
train = all_data_lab[:ntrain]
test = all_data_lab[ntrain:]

In [57]:
import numpy as np
from sklearn.model_selection import train_test_split

# DataFrameをndarrayに変換
X = np.array(train)
y = np.array(target)

# データを分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [58]:
# 標準化
from sklearn.preprocessing import QuantileTransformer

scaler = QuantileTransformer(output_distribution='normal')

X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

>本課題で検証した各手法の成績を記録するためのデータフレームを作成する。<br>
>各行が分割検証法、各列がパラメータチューニング法を示す。

In [79]:
# 各手法の結果を表に記載
summary = pd.DataFrame(index=['Holdout', 'Kfold', 'SSS'], columns=['Default', 'Grid', 'Random', 'Bayesian'])

summary

,Default,Grid,Random,Bayesian
Holdout,NaN,NaN,NaN,NaN
Kfold,NaN,NaN,NaN,NaN
SSS,NaN,NaN,NaN,NaN


>#### ベースラインモデルの学習（ホールドアウト検証）
>上記にて生成した抽出特徴量に対して学習を行う。以後はこのモデルをベースラインと呼ぶ。
>- 分類器：Logistic Regression (max iterations = 5000)
>- 標準化手法：Quantile Transformation (Gaussian Dist.)

In [59]:
# LogisticRegressionベースモデルの正解率を検証

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

lr = LogisticRegression(max_iter=5000)
lr.fit(X_train_std, y_train)

# 特徴量の重要性を取得
features_names = list(train.columns)

feature_importance_values = abs(lr.coef_.flatten())
feature_importances = pd.DataFrame({'feature': features_names, 'importance': feature_importance_values})

lr_pred = lr.predict_proba(X_val_std)

score = roc_auc_score(y_val, lr_pred[:,1])

print("ベースモデルのAUC ROC値：{:5f}".format(score))

ベースモデルのAUC ROC値：0.738888


>特徴量が多く、ハイパーパラメータチューニングに時間を要するため、重要性上位の特徴量抽出して学習を行う。

In [61]:
# 特徴量を重要性の高い順に表示
feature_importances.sort_values('importance', ascending = False, ignore_index=True).head(30)

,feature,importance
0,AMT_GOODS_PRICE^2,1.598194
1,AMT_GOODS_PRICE DTI,1.288114
2,AMT_ANNUITY DTI,1.050981
3,AMT_ANNUITY AMT_GOODS_PRICE,0.828995
4,AMT_ANNUITY EXT_SOURCE_3,0.471827
5,AMT_GOODS_PRICE EXT_SOURCE_3,0.341941
6,NAME_INCOME_TYPE_Maternity leave,0.326158
7,EXT_SOURCE_3^2,0.325988
8,AMT_GOODS_PRICE EXT_SOURCE_2,0.324959
9,NAME_INCOME_TYPE_Student,0.316060


In [230]:
# 上記表の10位までの特徴量を抽出
features_extract = feature_importances[feature_importances['importance'] >= 0.28]

train_ext = train[features_extract['feature']]
test_ext = test[features_extract['feature']]

train_ext

,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Student,AMT_ANNUITY,AMT_ANNUITY AMT_GOODS_PRICE,AMT_ANNUITY DTI,AMT_ANNUITY EXT_SOURCE_3,AMT_GOODS_PRICE^2,AMT_GOODS_PRICE DTI,AMT_GOODS_PRICE EXT_SOURCE_2,AMT_GOODS_PRICE EXT_SOURCE_3,EXT_SOURCE_3^2
0,0,0,24700.5,8.669875e+09,49595.855469,3443.355957,1.232010e+11,7.047689e+05,92291.750000,48930.906250,0.019434
1,0,0,35698.5,4.032145e+10,171022.593750,19051.982422,1.275770e+12,5.411152e+06,702628.437500,602804.437500,0.284827
2,0,0,6750.0,9.112500e+08,13500.000000,4924.072266,1.822500e+10,2.700000e+05,75080.562500,98481.445312,0.532159
3,0,0,29686.5,8.816891e+09,68758.882812,15843.429688,8.820900e+10,6.879015e+05,193166.015625,158506.343750,0.284827
4,0,0,21865.5,1.121700e+10,92321.000000,11669.429688,2.631690e+11,2.166000e+06,165572.750000,273783.687500,0.284827
...,...,...,...,...,...,...,...,...,...,...,...
307506,0,0,27558.0,6.200550e+09,44565.222656,14707.467773,5.062500e+10,3.638572e+05,153369.140625,120080.562500,0.284827
307507,0,0,12001.5,2.700337e+09,44930.617188,6405.097656,5.062500e+10,8.423438e+05,26092.529297,120080.562500,0.284827
307508,0,0,29979.0,1.753771e+10,132782.281250,6561.565918,3.422250e+11,2.591068e+06,313352.062500,128040.164062,0.047905
307509,0,0,20205.0,6.455498e+09,43731.062500,13358.188477,1.020802e+11,6.915157e+05,164274.171875,211231.937500,0.437097


In [247]:
# DataFrameをndarrayに変換
X = np.array(train_ext)
y = np.array(target)

# ホールドアウト法を関数化
def holdout_validate(model, test_size=0.25):
    # データを分割
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size)
    
    scaler = QuantileTransformer(output_distribution='normal')

    # 標準化
    X_train_std = scaler.fit_transform(X_train)
    X_val_std = scaler.transform(X_val)
    
    # 学習・推定
    model.fit(X_train_std, y_train)
    pred = model.predict_proba(X_val_std)
    
    return roc_auc_score(y_val, pred[:,1])

In [86]:
lr = LogisticRegression(max_iter=5000)

score = holdout_validate(lr)

summary['Default']['Holdout'] = score

print("特徴量を抽出したベースモデルのAUC ROC値：{:5f}".format(score))

特徴量を抽出したベースモデルのAUC ROC値：0.711301


>#### KFold交差検証

In [42]:
from sklearn.model_selection import KFold

# KFoldを実施
kf = KFold()

# 分割されたデータのインデックスを表示
for train, val in kf.split(X):
    print("TRAIN SET: {}\n Val SET: {}".format(train, val))

TRAIN SET: [ 61503  61504  61505 ... 307508 307509 307510]
 Val SET: [    0     1     2 ... 61500 61501 61502]
TRAIN SET: [     0      1      2 ... 307508 307509 307510]
 Val SET: [ 61503  61504  61505 ... 123002 123003 123004]
TRAIN SET: [     0      1      2 ... 307508 307509 307510]
 Val SET: [123005 123006 123007 ... 184504 184505 184506]
TRAIN SET: [     0      1      2 ... 307508 307509 307510]
 Val SET: [184507 184508 184509 ... 246006 246007 246008]
TRAIN SET: [     0      1      2 ... 246006 246007 246008]
 Val SET: [246009 246010 246011 ... 307508 307509 307510]


In [44]:
# KFold分割されたデータをを予測する関数を作成
def cross_validate(model, split_size=4):
    results = []
    kf = KFold(n_splits=split_size)
    
    for train_id, val_id in kf.split(X, y):
        train_X = X[train_id]
        train_y = y[train_id]
        val_X = X[val_id]
        val_y = y[val_id]
        
        # 標準化
        scaler = QuantileTransformer(output_distribution='normal')

        train_std = scaler.fit_transform(train_X)
        val_std = scaler.transform(val_X)

        # 学習、推定
        model.fit(train_std, train_y)
        pred = model.predict_proba(val_std)
        
        results.append(roc_auc_score(val_y, pred[:,1]))
    return np.array(results)

In [45]:
start = time.time()

result = cross_validate(lr).mean()

end = time.time()

print("Logistic Regressionの交差検証平均：{:5f}".format(result))
print("学習・推定にかかった時間：{:3f} sec".format(end - start))

Logistic Regressionの交差検証平均：0.714714
学習・推定にかかった時間：35.566313 sec


In [82]:
# cross_val_scoreを使用
# 標準化を行うため、Pipelineを併用

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

pipeline_lr = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                        ('clf', LogisticRegression(max_iter=5000))])

start = time.time()

score = cross_val_score(pipeline_lr, X, y, scoring='roc_auc', cv=4)

end = time.time()

summary['Default']['Kfold'] = score.mean()

print("LogisticRegressionの交差検証結果：{}".format(score))
print("平均：{:5f}".format(score.mean()))
print("学習・推定にかかった時間：{:3f} sec".format(end - start))

LogisticRegressionの交差検証結果：[0.71424229 0.7081932  0.71190598 0.71122861]
平均：0.711393
学習・推定にかかった時間：19.452269 sec


## 【問題2】グリッドサーチ
これまで分類器のパラメータには触れず、デフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになります。機械学習の前提として、パラメータは状況に応じて最適なものを選ぶ必要があります。最適なパラメータを探していくことを**パラメータチューニング**と呼びます。パラメータチューニングをある程度自動化する単純な方法としては**グリッドサーチ**があります。<br>
<br>
scikit-learnのGridSearchCVを使い、グリッドサーチを行うコードを作成してください。そして、ベースラインモデルに対して何らかしらのパラメータチューニングを行なってください。どのパラメータをチューニングするかは、使用した手法の公式ドキュメントを参考にしてください。<br>
<br>
[sklearn.model_selection.GridSearchCV — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)<br>
<br>
GridSearchCVクラスには引数としてモデル、探索範囲、さらにクロスバリデーションを何分割で行うかを与えます。クロスバリデーションの機能も含まれているため、これを使用する場合はKFoldクラスを利用する必要はありません。

>#### グリッドサーチ
>グリッドサーチにて探索したハイパーパラメータを実装したモデルを検証。

In [68]:
# LogisticRegressionのハイパーパラメータチューニング
from sklearn.model_selection import GridSearchCV

pipeline_lr = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                        ('clf', LogisticRegression(max_iter=5000))])

# LogisticRegressionのparameter gridを設定
param_grid = { 
    'clf__penalty': ['none', 'l2'],
    'clf__C': [10**i for i in range(-4,4)]
}

# Grid search実施
start = time.time()
grid_search = GridSearchCV(pipeline_lr, param_grid, cv=4, scoring='roc_auc')
grid_search.fit(X, y)
end = time.time()

print('Best parameters: {}'.format(grid_search.best_params_))
print('Best score: {}'.format(grid_search.best_score_))
print("探索・学習・推定にかかった時間：{:3f} sec".format(end - start))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore t

Best parameters: {'clf__C': 0.01, 'clf__penalty': 'none'}
Best score: 0.7115211582186709
探索・学習・推定にかかった時間：356.706152 sec


In [83]:
# grid searchで得られたハイパーパラメータを設定
lr_grid = LogisticRegression(C=0.01, penalty='none', max_iter=5000)

pipe_lr_grid = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                          ('clf', lr_grid)])

score = cross_val_score(pipe_lr_tuned, X, y, scoring='roc_auc', cv=4).mean()b

summary['Grid']['Kfold'] = score

print("チューニング後のvalidation ROC AUC値：{:5f}".format(score))

pipe_lr_tuned.fit(X_train, y_train)
pipe_lr_tuned_pred = pipe_lr_tuned.predict_proba(X_val)

print("検証用データに対するチューニング後のROC AUC値：{:5f}".
      format(roc_auc_score(y_val, pipe_lr_tuned_pred[:,1])))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore t

チューニング後のvalidation ROC AUC値：0.711516


/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


検証用データに対するチューニング後のROC AUC値：0.713966


In [87]:
# ホールドアウト法のスコアも検証
score = holdout_validate(lr_grid)
summary['Grid']['Holdout'] = score
print("チューニング後のholdout ROC AUC値：{:5f}".format(score))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "


チューニング後のholdout ROC AUC値：0.709846


## 【問題3】Kaggle Notebooksからの調査
KaggleのNotebooksから様々なアイデアを見つけ出して、列挙してください。

- Logistic Regressionのハイパーパラメータを調査
- 層化シャッフル分割交差検証を実装
- random search実装
- bayesian optimization実装
- 別の分類器を試す

Logistic Regressionの仕組みを改めて理解する<br>
[Qiita - ロジスティック回帰の初歩を理解する](https://qiita.com/iwashi-kun/items/030ed4dc48b16ca1b5a1)<br>
<br>
パラメータについて調査<br>
[Don’t Sweat the Solver Stuff](https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451)<br>
[Discussion - Stack Overflow](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions/52388406#52388406)<br>
<br>
様々なcross validation手法について<br>
[交差検証（cross validation／クロスバリデーション）の種類を整理してみた](https://aizine.ai/cross-validation0910/#toc9)

## 【問題4】高い汎化性能のモデル作成
問題3で見つけたアイデアと、独自のアイデアを組み合わせ高い汎化性能のモデル作りを進めてください。<br>
<br>
その過程として、何を行うことで、クロスバリデーションの結果がどの程度変化したかを表にまとめてください。

>#### 層化シャッフル分割交差検証
>KFoldでの交差検証は時間がかかりすぎることが判明。また、このHome Creditデータはクラスが不均衡なため、別の交差検証手法を採用。時間短縮を図ったShuffleSplit、および不均衡に対応するためのStratifiedKFoldを合わせたStratifiedShuffleSplitを採用。

In [73]:
# 層化シャッフル分割交差検証を読み込む
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit()

In [74]:
# 分割されたデータのインデックスを表示
for train, val in sss.split(X, y):
    print("TRAIN sample size: {}\n VAL sample size: {}".format(len(train), len(val)))

TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752
TRAIN sample size: 276759
 VAL sample size: 30752


In [75]:
# 層化シャッフル分割分割されたデータを検証する関数を作成

def sss_cross_validate(model, split_size=4):
    results = []
    sss = StratifiedShuffleSplit(n_splits=split_size)
    
    for train_id, val_id in sss.split(X, y):
        train_X = X[train_id]
        train_y = y[train_id]
        val_X = X[val_id]
        val_y = y[val_id]
        
        # 標準化
        scaler = QuantileTransformer(output_distribution='normal')

        train_std = scaler.fit_transform(train_X)
        val_std = scaler.transform(val_X)
        
        # 学習、推定
        model.fit(train_std, train_y)
        pred = model.predict_proba(val_std)
        
        results.append(roc_auc_score(val_y, pred[:,1]))
    return np.array(results)

In [89]:
# 層化シャッフル分割のスコアを検証
start = time.time()

score = sss_cross_validate(lr_grid).mean()

end = time.time()

summary['Grid']['SSS'] = score

print("Logistic Regressionの層化シャッフル分割交差検証平均：{:5f}".format(score))
print("学習・推定にかかった時間：{:3f} sec".format(end - start))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio "
/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1505: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore t

Logistic Regressionの層化シャッフル分割交差検証平均：0.712477
学習・推定にかかった時間：20.752684 sec


In [90]:
# デフォルトパラメータの場合も検証
score = sss_cross_validate(lr).mean()

summary['Default']['SSS'] = score
print("デフォルトパラメータ＋層化シャッフル分割交差検証平均：{:5f}".format(score))

デフォルトパラメータ＋層化シャッフル分割交差検証平均：0.708310


>#### Random Search
>Random Searchによる最適パラメータ探索を実装。Grid Searchとの違いを検証。

In [77]:
# Random Searchでハイパーパラメータを探索
from sklearn.model_selection import RandomizedSearchCV

param_grid = { 
    'clf__penalty': ['l1', 'l2'],
    'clf__C': [10**i for i in range(-4,4)],
    'clf__solver': ['lbfgs', 'liblinear', 'saga']
}


start = time.time()
rand_search = RandomizedSearchCV(pipeline_lr, param_grid, n_iter=10, cv=4,
                                 n_jobs=-1, scoring='roc_auc')
rand_search.fit(X, y)
end = time.time()

print('Best parameters: {}'.format(rand_search.best_params_))
print("探索・学習・推定にかかった時間：{:3f} sec".format(end - start))

Best parameters: {'clf__solver': 'liblinear', 'clf__penalty': 'l1', 'clf__C': 100}
探索・学習・推定にかかった時間：375.993986 sec


In [158]:
# RandomizedSearchのスコア
lr_rand = LogisticRegression(solver='liblinear', penalty='l1', C=100, max_iter=5000)
pipe_lr_rand = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                          ('clf', lr_rand)])

score_randh = holdout_validate(lr_rand)
summary['Random']['Holdout'] = score_randh
print("Random Searchパラメータ＋ホールドアウト分割交差検証平均：{:5f}".format(score_randh))

score_rands = sss_cross_validate(lr_rand).mean()
summary['Random']['SSS'] = score_rands
print("Random Searchパラメータ＋層化シャッフル分割交差検証平均：{:5f}".format(score_rands))

score_randk = cross_val_score(pipe_lr_rand, X, y, scoring='roc_auc', cv=4).mean()
summary['Random']['Kfold'] = score_randk
print("Random Searchパラメータ＋KFold分割交差検証平均：{:5f}".format(score_randk))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


Random Searchパラメータ＋ホールドアウト分割交差検証平均：0.711193


/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Random Searchパラメータ＋層化シャッフル分割交差検証平均：0.710201
Random Searchパラメータ＋KFold分割交差検証平均：0.711378


/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


>#### Bayesian Optimization
>Bayesian最適化はGridやRandom Searchのように無作為にパラメータを選んで検証するのではなく、イテレーション毎に最も高かったスコアのパラメータ組み合わせを参照し、パラメータを微調整する。ここではhyperoptライブラリを使用。

In [138]:
# hyperopt実装
from hyperopt import hp, tpe, Trials, fmin

hyperopt_params = {
    'C': hp.loguniform('C', -4, 4),
    'warm_start': hp.choice('warm_start', [True, False]),
    'solver': hp.choice('solver', ['liblinear', 'lbfgs', 'saga']),
    'fit_intercept': hp.choice('fit_intercept', [True, False]),
    'class_weight': hp.choice('class_weight', [None, 'balanced'])
}


def objective(params, X=X, y=y, cv=4):
    # モデルのインスタンス化
    classifier = LogisticRegression(**params, max_iter=5000)
    # cross validationを計算
    scores = cross_validate(classifier)
    # scoresの平均値を取得
    score = scores.mean()
    # roc aucを最大化したいので、1から引いた値を出力
    loss = 1 - score
    return loss


trials = Trials()
n_iter = 50

start = time.time()

best = fmin(fn=objective,
          space=hyperopt_params, 
          algo=tpe.suggest,
          max_evals=n_iter, 
          trials=trials,
          verbose=1
         )

end = time.time()

print("Best params {}".format(best))
print("探索にかかった時間：{:3f} sec".format(end - start))

 34%|███▍      | 17/50 [17:36<1:13:08, 132.98s/trial, best loss: 0.2878330416557766]

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



 58%|█████▊    | 29/50 [34:15<13:22, 38.23s/trial, best loss: 0.2878330416557766]   

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)



100%|██████████| 50/50 [54:49<00:00, 65.79s/trial, best loss: 0.2878089898036611]   
Best params {'C': 1.650775575860084, 'class_weight': 1, 'fit_intercept': 0, 'solver': 1, 'warm_start': 1}
探索にかかった時間：3289.646760 sec


In [156]:
# Bayesian Optimizationで得られたハイパーパラメータを設定
lr_bayes = LogisticRegression(C=1.650775575860084, warm_start=False, fit_intercept=True, class_weight='balanced', solver='lbfgs', max_iter=5000)

pipe_lr_bayes = Pipeline([('scaler', QuantileTransformer(output_distribution='normal')),
                          ('clf', lr_bayes)])

score_bayes_h = holdout_validate(lr_bayes)
summary['Bayesian']['Holdout'] = score_bayes_h
print("Bayesian Searchパラメータ＋ホールドアウト分割交差検証平均：{:5f}".format(score_bayes_h))

score_bayes_k = cross_val_score(pipe_lr_bayes, X, y, scoring='roc_auc', cv=4).mean()
summary['Bayesian']['Kfold'] = score_bayes_k
print("Bayesian Searchパラメータ＋KFold分割交差検証平均：{:5f}".format(score_bayes_k))

score_bayes_s = sss_cross_validate(lr_bayes).mean()
summary['Bayesian']['SSS'] = score_bayes_s
print("Bayesian Searchパラメータ＋層化シャッフル分割交差検証平均：{:5f}".format(score_bayes_s))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Bayesian Searchパラメータ＋ホールドアウト分割交差検証平均：0.714919


/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Bayesian Searchパラメータ＋KFold分割交差検証平均：0.712109
Bayesian Searchパラメータ＋層化シャッフル分割交差検証平均：0.709537


/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


>#### 各手法の成績
>ここまでで検証した各手法のスコアを確認する。

In [160]:
# 各手法の平均スコアを表示
summary

,Default,Grid,Random,Bayesian
Holdout,0.711301,0.709846,0.711193,0.714919
Kfold,0.711393,0.711516,0.711378,0.712109
SSS,0.70831,0.712477,0.710201,0.709537


>パラメータチューニング手法の中では、Bayesian最適化が最もスコアが高く、最適パラメータを探索する精度が高いことが窺える。また、学習時間もGridやRandom Searchよりも短いため、より多くのパラメータを探索できる。分割手法はHoldoutが最もスコアが高いが、過学習の恐れがあるため、KfoldまたはStratified Shuffle Splitを適用することが望ましい。

>#### lightGBMの検証
>分類器をlightGBMに変更し、Bayesian Optimization＋KFold交差検証の学習を調査する。

In [221]:
import lightgbm as lgb

X = train_ext
y = target

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25)

lgb_train = lgb.Dataset(data=train_X, label=train_y)
lgb_val = lgb.Dataset(data=val_X, label=val_y, reference=lgb_train)

In [222]:
params = {'task': 'train', 'boosting_type': 'gbdt', 
          'objective': 'binary', 'metric': 'auc', 'n_estimators':5000}

print("ベースラインlightGBMのHoldout検証")
model = lgb.train(params, lgb_train, valid_sets=lgb_val,
                  early_stopping_rounds=150, verbose_eval=200)

ベースラインlightGBMのHoldout検証
Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.726042
Early stopping, best iteration is:
[137]	valid_0's auc: 0.727282


In [223]:
lgb_train_cv = lgb.Dataset(data=X, label=y)
scores = lgb.cv(params, lgb_train_cv, num_boost_round=5000,
                    early_stopping_rounds=150, nfold=4)

print("ベースラインlightGBMの交差検証ベストスコア：{:5f}".format(max(scores['auc-mean'])))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/lightgbm/engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


ベースラインlightGBMの交差検証ベストスコア：0.724505


In [219]:
from hyperopt import STATUS_OK

lgb_params = {
    'boosting_type': hp.choice('boosting_type', 
                [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
#                  {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                 {'boosting_type': 'goss', 'subsample': 1.0}
                ]),
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'num_leaves': hp.quniform('num_leaves', 30, 151, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

lgb_train = lgb.Dataset(X, y)

def objective(params, n_folds=4):
    
    # nested spaceの取得
    params['subsample'] = params['boosting_type']['subsample']
    boost_type = params['boosting_type']['boosting_type']
    del params['boosting_type']
    params['boosting_type'] = boost_type
    
    # 数値型への変換
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    
    # cross validationを計算
    scores = lgb.cv(params, lgb_train, num_boost_round=500,
                    early_stopping_rounds=150, nfold=n_folds, metrics='auc')
    
    # scoresの平均値を取得
    best_score = max(scores['auc-mean'])
    # roc aucを最大化したいので、1から引いた値を出力
    loss = 1 - best_score
    
    return {'loss': loss, 'params': params, 'status': STATUS_OK}


trials = Trials()
n_iter = 100

start = time.time()

best = fmin(fn=objective,
          space=lgb_params, 
          algo=tpe.suggest,
          max_evals=n_iter, 
          trials=trials,
          verbose=1
         )

end = time.time()

print("Best params {}".format(best))
print("探索にかかった時間：{:3f} sec".format(end - start))

100%|██████████| 100/100 [54:14<00:00, 32.54s/trial, best loss: 0.2735315255959847] 
Best params {'boosting_type': 0, 'class_weight': 0, 'colsample_by_tree': 0.8011473419486966, 'gdbt_subsample': 0.7792465808431693, 'learning_rate': 0.023940707008353938, 'min_child_samples': 435.0, 'num_leaves': 33.0, 'reg_alpha': 0.340140470130553, 'reg_lambda': 0.07433696526494404, 'subsample_for_bin': 180000.0}
探索にかかった時間：3254.222796 sec


In [224]:
# チューニングしたモデルの検証
params = {'task': 'train', 'boosting_type': 'gbdt', 
          'objective': 'binary', 'metric': 'auc', 'n_estimators':5000,
          'class_weight': None, 'colsample_by_tree': 0.6127854826784527, 'gdbt_subsample': 0.9903813589683813,
          'learning_rate': 0.010170991958005731, 'min_child_samples': 485, 'num_leaves': 53, 
          'reg_alpha': 0.9983892325538006, 'reg_lambda': 0.6735746969077508, 'subsample_for_bin': 180000}

lgb_train_cv = lgb.Dataset(data=X, label=y)
scores = lgb.cv(params, lgb_train_cv, num_boost_round=5000,
                    early_stopping_rounds=150, nfold=4)

print("チューニング後lightGBMの交差検証ベストスコア：{:5f}".format(max(scores['auc-mean'])))

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/lightgbm/engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


チューニング後lightGBMの交差検証ベストスコア：0.725873


>lightGBMはロジスティック回帰より良いスコアを出したが、パラメータチューニングによる改善は見られなかった。

## 【問題5】最終的なモデルの選定
最終的にこれは良いというモデルを選び、推定した結果をKaggleに提出してスコアを確認してください。どういったアイデアを取り入れ、どの程度のスコアになったかを記載してください。

>Bayesian最適化で得られたハイパーパラメータを実装したlightGBMを提出用スコアに使用。

In [251]:
# Submission用DF作成
params = {'task': 'train', 'boosting_type': 'gbdt', 
          'objective': 'binary', 'metric': 'auc', 'n_estimators':5000,
          'class_weight': None, 'colsample_by_tree': 0.6127854826784527, 'gdbt_subsample': 0.9903813589683813,
          'learning_rate': 0.010170991958005731, 'min_child_samples': 485, 'num_leaves': 53, 
          'reg_alpha': 0.9983892325538006, 'reg_lambda': 0.6735746969077508, 'subsample_for_bin': 180000}

X = train_ext
y = target

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25, stratify=y)

lgb_train = lgb.Dataset(data=train_X, label=train_y)
lgb_val = lgb.Dataset(data=val_X, label=val_y, reference=lgb_train)

model_submit = lgb.train(params, lgb_train, valid_sets=lgb_val,
                  early_stopping_rounds=150, verbose_eval=200)

/Users/tamiyagt/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 150 rounds
[200]	valid_0's auc: 0.724419
[400]	valid_0's auc: 0.729092
[600]	valid_0's auc: 0.729959
[800]	valid_0's auc: 0.73014
Early stopping, best iteration is:
[839]	valid_0's auc: 0.730205


In [252]:
pred_submit = model_submit.predict(test_ext)
output = pd.DataFrame({'SK_ID_CURR': sk_id,
                       'TARGET': pred_submit})

output

,SK_ID_CURR,TARGET
0,100001,0.067804
1,100005,0.122863
2,100013,0.030293
3,100028,0.035472
4,100038,0.107270
...,...,...
48739,456221,0.037530
48740,456222,0.048289
48741,456223,0.093389
48742,456224,0.071363


In [253]:
# csvに書き込み
output.to_csv('submission1.csv', index=False)

>Kaggle上のスコアは0.70411。